Data from [https://sci-hub.se/https://onlinelibrary.wiley.com/doi/10.1111/jiec.12302] and [https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1111%2Fjiec.12302&file=jiec12302-sup-0001-SupMat.pdf].
More details also here: https://link.springer.com/article/10.1186/s40008-015-0016-5

In [78]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX
using DataFrames
using Revise

In [79]:
# Load the workbook
xf = XLSX.readxlsx("../data/WTMBT-IO.xlsx");

In [80]:
dict = Dict()
for i in XLSX.sheetnames(xf)
    if i == "PARAMS"
        dict[i] = DataFrame(XLSX.gettable(xf[i], header=true))
    else
        dict[i] = DataFrame(XLSX.gettable(xf[i], header=false))
        dict[i] = Matrix{Float64}(dict[i])
    end
end

In [81]:
REGIONS = dict["PARAMS"][1,"Value"]
SECTORS = dict["PARAMS"][2,"Value"]
FACTORS = dict["PARAMS"][3,"Value"]
DEMANDS = dict["PARAMS"][4,"Value"]
VARIABLES = sort(filter(!=("PARAMS"), XLSX.sheetnames(xf)));

In [82]:
dict_reg = Dict()
idx_sec = 1
idx_fac = 1
idx_sec2 = 1

for reg in 1:REGIONS
    for var in VARIABLES
        if var == "A"
            dict_reg[var * string(reg)] = 
            dict[var][idx_sec:(reg*SECTORS),idx_sec:(reg*SECTORS)]
        elseif var == "R"
            dict_reg[var * string(reg)] = 
            dict[var][idx_fac:(reg*FACTORS),idx_sec:(reg*SECTORS)]
        elseif var == "T"
            for reg2 in 1:REGIONS
                if reg != reg2
                    dict_reg[var * string(reg) * string(reg2)] = 
                    dict[var][idx_sec:(reg*SECTORS),idx_sec2:(reg2*SECTORS)]
                end
                idx_sec2 += SECTORS
            end
            idx_sec2 = 1
        elseif var == "Y"
            dict_reg[var * string(reg)] =
            dict[var][idx_sec:(reg*SECTORS),1:DEMANDS]
            dict_reg[lowercase(var) * string(reg)] =
            vec(sum(dict_reg[var * string(reg)], dims=2))
        elseif var in ["pii", "phi"]
            dict_reg[var * string(reg)] =
            dict[var][idx_fac:(reg*FACTORS)]
        end
    end
    idx_sec += SECTORS
    idx_fac += FACTORS
end


In [83]:
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));

# Define the optimisation model
@variable(primal, x[i=1:REGIONS, j=1:SECTORS] >= 0)
@variable(primal, e[i=1:REGIONS, k=1:REGIONS, j=1:SECTORS] >= 0)

@objective(primal, Min, sum(dict_reg["pii"*string(i)]'*dict_reg["R"*string(i)]*x[i,:] for i=1:REGIONS))

for i=1:REGIONS
    primal[Symbol("con"*string(i))] = @constraints(primal, begin
        # Goods balance
        x[i,:] - dict_reg["A"*string(i)]*x[i,:] + 
        sum(e[i,k,:] for k=1:REGIONS) - sum(e[k,i,:] for k=1:REGIONS) - 
        sum(dict_reg["T"*string(k,i)]*e[k,i,:] for k=(1:REGIONS)[Not(i)]) .>= dict_reg["y"*string(i)],
        (base_name = "goods"*string(i))

        # Factor constraint,
        dict_reg["R"*string(i)]*x[i,:] .<= dict_reg["phi"*string(i)],
        (base_name = "factors"*string(i))
    end)
end

# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(x)
@show value.(e);

termination_status(primal) = MathOptInterface.OPTIMAL
primal_status(primal) = MathOptInterface.FEASIBLE_POINT
dual_status(primal) = MathOptInterface.FEASIBLE_POINT
objective_value(primal) = 1438.628044466307
value.(x) = [0.0 164.51661015159834 0.0 13.533492502987041; 78.95166101515983 0.0 0.0 0.0; 0.0 0.0 98.85480157895263 0.0]
value.(e) = [0.0 23.16132881212787 0.0; 0.0 0.0 0.0; 0.0 10.0 0.0;;; 0.0 0.0 0.0; 17.895166101515986 0.0 0.0; 41.656440473685784 0.0 0.0;;; 0.0 0.0 66.12172929697303; 0.0 0.0 20.79033220303197; 0.0 0.0 0.0;;; 0.0 0.0 0.0; 8.710702894264362 0.0 0.0; 0.0 6.922636453657968 0.0]


In [ ]:
typeof(dict_reg["y1"]'*p[1,:])

In [ ]:
typeof(dict_reg["phi1"]'*r[1,:])

In [84]:
# Instantiate the model
dual = Model(GLPK.Optimizer)
(@isdefined dual) && (dual = nothing; dual = Model(GLPK.Optimizer));

# Define the optimisation model
@variable(dual, p[i=1:REGIONS, j=1:SECTORS] >= 0)
@variable(dual, r[i=1:REGIONS, k=1:FACTORS] >= 0)

@objective(dual, Max, sum(vec(dict_reg["y"*string(i)])'*p[i,:]-
dict_reg["phi"*string(i)]'*r[i,:] for i=1:REGIONS))

for i in 1:REGIONS
    dual[Symbol("con"*string(i))] = @constraints(dual, begin
        # Zero profit
        (I - dict_reg["A"*string(i)]')*p[i,:] .<= 
        dict_reg["R"*string(i)]'*(dict_reg["pii"*string(i)]+r[i,:]),
        (base_name = "zero_profit"*string(i))

        # Import price formation,
        #for j in 1:REGIONS
        #    if i != j
        #        p[i,:] - p[j,:] .<= dict_reg["T"*string(j)*string(i)]'*p[i,:],
        #        (base_name = "import_price"*string(i)*string(j))
        #    end
        #end
        [j=(1:REGIONS)[Not(i)]], p[i,:] - p[j,:] .<= dict_reg["T"*string(j)*string(i)]'*p[i,:], (base_name = "import_price"*string(i)*string(j))
    end)
end

# Run the model and show results
optimize!(dual)
@show termination_status(dual)
@show primal_status(dual)
@show dual_status(dual)
@show objective_value(dual)
@show value.(p)
@show value.(r);

termination_status(dual) = MathOptInterface.OPTIMAL
primal_status(dual) = MathOptInterface.FEASIBLE_POINT
dual_status(dual) = MathOptInterface.FEASIBLE_POINT
objective_value(dual) = 1438.6280444663073
value.(p) = [18.16547219841851 9.05327227986224 6.911327880192132 5.666318396068514; 17.882156278615085 9.393251383626351 6.831999422647172 5.666318396068514; 18.2391343375674 9.563240935508407 6.4410234533184445 5.666318396068514]
value.(r) = [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]


In [ ]:
primal[:con2][2]

In [ ]:
all_constraints(primal, VariableRef, MOI.GreaterThan{Float64})

In [ ]:
all_constraints(primal; include_variable_in_set_constraints = false)

In [ ]:
solution_summary(primal; verbose = true)

In [ ]:
value.(e)

In [ ]:
value.(e)[3,1,:]

# Set up MRIOT

In [85]:
dict_mrio = deepcopy(dict_reg);

In [86]:
for i in 1:REGIONS
    dict_mrio["x"*string(i)] = value.(x)[i,:]
    dict_mrio["p"*string(i)] = value.(p)[i,:]
    dict_mrio["r"*string(i)] = value.(r)[i,:]
    for j in (1:REGIONS)[Not(i)]
        dict_mrio["e"*string(i)*string(j)] = value.(e)[i,j,:]
    end
end

## (I think something's still wrong with the S-matrices. The Big matrices calculated later seem a little off...)

In [87]:
for j in 1:REGIONS
    for i in 1:REGIONS
        if i == j
            foo = value.(x)[j,:] + vec(sum(value.(e)[:,j,:], dims = 1))
            bar = value.(x)[j,:] / foo
            replace!(bar, Inf=>0)

            dict_mrio["S"*string(i)*string(j)] = Diagonal(bar)
        else
            foo = value.(x)[j,:] + vec(sum(value.(e)[:,j,:], dims = 1))
            bar = value.(e)[i,j,:] / foo
            replace!(bar, Inf=>0)
            
            dict_mrio["S"*string(i)*string(j)] = Diagonal(bar)
        end
    end
end

In [ ]:
dict_mrio["S22"]

In [88]:
for j in 1: REGIONS
    dict_mrio["T_S"*string(j)] =
        sum(
        dict_mrio["T"*string(i)*string(j)]*
        dict_mrio["S"*string(i)*string(j)] 
        for i=(1:REGIONS)[Not(j)]
        )
end

In [ ]:
dict_mrio["T_S1"]

In [95]:
dict_mrio["T_S"] = zeros(REGIONS*SECTORS, REGIONS*SECTORS)
dict_mrio["S_B"] = zeros(REGIONS*SECTORS, REGIONS*SECTORS)
dict_mrio["A"] = zeros(REGIONS*SECTORS, REGIONS*SECTORS)
dict_mrio["R"] = zeros(REGIONS*FACTORS, REGIONS*SECTORS)
dict_mrio["y"] = zeros(REGIONS*SECTORS, DEMANDS)
dict_mrio["p"] = zeros(REGIONS*SECTORS, 1)
dict_mrio["pii"] = zeros(REGIONS*FACTORS, 1)
dict_mrio["r"] = zeros(REGIONS*FACTORS, 1);

In [96]:
idx_sec = 1
idx_fac = 1

for i in 1:REGIONS
    idx_reg = 1
    
    dict_mrio["T_S"][idx_sec:i*SECTORS,idx_sec:i*SECTORS] = 
        dict_mrio["T_S"*string(i)]
    dict_mrio["A"][idx_sec:i*SECTORS,idx_sec:i*SECTORS] = 
        dict_mrio["A"*string(i)]
    dict_mrio["R"][idx_fac:i*FACTORS,idx_sec:i*SECTORS] = 
        dict_mrio["R"*string(i)]
    dict_mrio["y"][idx_sec:i*SECTORS,:] = 
        dict_mrio["y"*string(i)]
    dict_mrio["p"][idx_sec:i*SECTORS,:] = 
        dict_mrio["p"*string(i)]
    dict_mrio["pii"][idx_fac:i*FACTORS,:] = 
        dict_mrio["pii"*string(i)]
    dict_mrio["r"][idx_fac:i*FACTORS,:] = 
        dict_mrio["r"*string(i)]
    for j in 1:REGIONS
        dict_mrio["S_B"][idx_sec:i*SECTORS,idx_reg:j*SECTORS] = 
            Matrix(dict_mrio["S"*string(i)*string(j)])
        idx_reg += SECTORS
    end
    idx_sec += SECTORS
    idx_fac += FACTORS
end

In [91]:
dict_mrio["A_B"] = dict_mrio["S_B"]*(I + dict_mrio["T_S"])*dict_mrio["A"]

12×12 Matrix{Float64}:
 0.0         0.0         0.0         …  0.0        0.0         0.0
 0.181765    0.36353     0.254471       0.0        0.0         0.0
 0.0         0.0         0.0            0.106782   0.017797    0.17797
 3.16378e-5  6.32755e-5  4.42929e-5     0.0        0.0         0.0
 0.0         0.0         0.0            0.0        0.0         0.0
 0.0197713   0.0395427   0.0276799   …  0.0        0.0         0.0
 0.0         0.0         0.0            0.0335749  0.00559581  0.0559581
 2.03633e-5  4.07267e-5  2.85087e-5     0.0        0.0         0.0
 0.0         0.0         0.0            0.0        0.0         0.0
 0.0460238   0.0920477   0.0644334      0.0        0.0         0.0
 0.0         0.0         0.0         …  0.159643   0.0266072   0.266072
 0.0         0.0         0.0            0.0        0.0         0.0

In [92]:
dict_mrio["y_B"] = dict_mrio["S_B"]*(I + dict_mrio["T_S"])*Diagonal(vec(dict_mrio["y"]))

12×12 Matrix{Float64}:
 0.0   0.0         0.0  0.0   6.17413    …  0.0  0.0  0.0  0.0  0.0       0.0
 0.0  14.5412      0.0  0.0   0.0           0.0  0.0  0.0  0.0  0.0       0.0
 0.0   0.0         0.0  0.0   0.0           0.0  0.0  0.0  0.0  2.49158   0.0
 0.0   0.00253102  0.0  0.0   0.0           0.0  0.0  0.0  0.0  0.0       0.0
 0.0   0.0         0.0  0.0  21.0462        0.0  0.0  0.0  0.0  0.0       0.0
 0.0   1.58171     0.0  0.0   0.0        …  0.0  0.0  0.0  0.0  0.0       0.0
 0.0   0.0         0.0  0.0   0.0           0.0  0.0  0.0  0.0  0.783414  0.0
 0.0   0.00162907  0.0  0.0   0.0           0.0  0.0  0.0  0.0  0.0       0.0
 0.0   0.0         0.0  0.0   2.66571       0.0  0.0  0.0  0.0  0.0       0.0
 0.0   3.68191     0.0  0.0   0.0           0.0  0.0  0.0  0.0  0.0       0.0
 0.0   0.0         0.0  0.0   0.0        …  0.0  0.0  0.0  0.0  3.72501   0.0
 0.0   0.0         0.0  0.0   0.0018104     0.0  0.0  0.0  0.0  0.0       0.0

In [106]:
dict_mrio["Phi"] = Diagonal(vec(dict_mrio["pii"]+dict_mrio["r"]))*dict_mrio["R"]*inv(I - dict_mrio["A_B"])*dict_mrio["S_B"]*(I + dict_mrio["T_S"])*inv(Diagonal(vec(dict_mrio["p"])))

9×12 Matrix{Float64}:
 0.0  0.132454    0.0  0.000670455  0.0523701   …  0.0  0.0  0.080599     0.0
 0.0  0.0141514   0.0  2.68497e-5   0.00466391     0.0  0.0  0.227362     0.0
 0.0  0.00332549  0.0  6.30953e-6   0.133985       0.0  0.0  0.000598221  0.0
 0.0  0.0746242   0.0  0.00351383   0.185214       0.0  0.0  0.0223494    0.0
 0.0  0.0134835   0.0  2.55825e-5   0.00444379     0.0  0.0  0.216631     0.0
 0.0  0.00906866  0.0  1.72062e-5   0.365378    …  0.0  0.0  0.00163136   0.0
 0.0  0.326135    0.0  0.000618785  0.0692289      0.0  0.0  0.0798875    0.0
 0.0  0.0160279   0.0  3.04102e-5   0.00528238     0.0  0.0  0.257512     0.0
 0.0  0.00143579  0.0  2.72416e-6   0.0578484      0.0  0.0  0.000258285  0.0

In [112]:
dict_mrio["S_B"]

12×12 Matrix{Float64}:
 0.0  0.0        0.0  0.0         …  0.0         0.0  0.0  0.0       0.0
 0.0  0.72706    0.0  0.0            0.0         0.0  0.0  0.0       0.0
 0.0  0.0        0.0  0.0            0.0         0.0  0.0  0.355939  0.0
 0.0  0.0        0.0  0.00593754     0.0         0.0  0.0  0.0       0.0
 0.0  0.0        0.0  0.0            0.0         0.0  0.0  0.0       0.0
 0.0  0.0790854  0.0  0.0         …  0.0         0.0  0.0  0.0       0.0
 0.0  0.0        0.0  0.0            0.0         0.0  0.0  0.111916  0.0
 0.0  0.0        0.0  0.00382164     0.0         0.0  0.0  0.0       0.0
 0.0  0.0        0.0  0.0            0.0         0.0  0.0  0.0       0.0
 0.0  0.184095   0.0  0.0            0.0         0.0  0.0  0.0       0.0
 0.0  0.0        0.0  0.0         …  0.0         0.0  0.0  0.532144  0.0
 0.0  0.0        0.0  0.0            0.00379821  0.0  0.0  0.0       0.0

In [102]:
Diagonal(vec(dict_mrio["pii"]+dict_mrio["r"]))

9×9 Diagonal{Float64, Vector{Float64}}:
 2.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   2.2   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   5.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   0.5   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   5.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   2.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   2.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   2.0

# Scenarios
Change RHS of goods balance (iteratively?)

In [ ]:
# Get data from all worksheets and convert it to float64 data type
A = convert(Matrix{Float64}, xf["A!A1:L12"])
R = convert(Matrix{Float64}, xf["R!A1:L9"])
T = convert(Matrix{Float64}, xf["T!A1:L12"])
y = convert(Matrix{Float64}, xf["y!A1:A12"])
pii = convert(Matrix{Float64}, xf["pii!A1:A9"])
phi = convert(Matrix{Float64}, xf["phi!A1:A9"]);

In [ ]:
dict_reg = Dict()
dict_reg["A_1"] = dict["A"][1:4,1:4]
dict_reg["A_2"] = dict["A"][5:8,5:8]
dict_reg["A_3"] = dict["A"][9:12,9:12]
dict_reg["R_1"] = dict["R"][1:3,1:3]
dict_reg["R_2"] = dict["R"][4:6,4:6]
dict_reg["R_3"] = dict["R"][7:9,7:9]
dict_reg["T_12"] = dict["T"][1:4,5:8]
dict_reg["T_13"] = dict["T"][1:4,9:12]
dict_reg["T_23"] = dict["T"][5:8,9:12]
dict_reg["T_21"] = deepcopy(dict_reg["T_12"])
dict_reg["T_31"] = deepcopy(dict_reg["T_13"])
dict_reg["T_32"] = deepcopy(dict_reg["T_23"])
dict_reg["y_1"] = dict["y"][1:4]
dict_reg["y_2"] = dict["y"][5:8]
dict_reg["y_3"] = dict["y"][9:12]
dict_reg["pii_1"] = dict["pii"][1:3]
dict_reg["pii_2"] = dict["pii"][4:6]
dict_reg["pii_3"] = dict["pii"][7:9]
dict_reg["phi_1"] = dict["phi"][1:3]
dict_reg["phi_2"] = dict["phi"][4:6]
dict_reg["phi_3"] = dict["phi"][7:9];

In [ ]:
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));

# Define the optimisation model
@variable(primal, x[i=1:REGIONS, j=1:SECTORS] >= 0)
@variable(primal, e[i=1:REGIONS, k=1:REGIONS, j=1:SECTORS] >= 0)

@objective(primal, Min, sum(dict_reg["pii"*string(i)]'*dict_reg["R"*string(i)]*x[i,:] for i=1:REGIONS))

for i=1:REGIONS#, j=1:SECTORS, k=1:REGIONS, l=1:FACTORS
    @constraints(primal, begin
        #goods*i,
        x[i,:] - dict_reg["A"*string(i)]*x[i,:] + 
        sum(e[i,k,:] for k=1:REGIONS) - sum(e[k,i,:] for k=1:REGIONS) - 
        sum(dict_reg["T"*string(k,i)]*e[k,i,:] for k=(1:REGIONS)[Not(i)]) .>= dict_reg["y"*string(i)],
        (base_name = "goods"*string(i))

        #factors*i,
        dict_reg["R"*string(i)]*x[i,:] .<= dict_reg["phi"*string(i)],
        (base_name = "factors"*string(i))
    end)
end

# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(x)
@show value.(e);